In [55]:
import sys
sys.path.append('..')

In [56]:
import pandas as pd
import random
import numpy as np
from continuous_BCQ.utils import ReplayBuffer

In [57]:
df = pd.read_csv("df_thickener1_20230201_1min.csv")

In [58]:
df

,time,feed_c,out_c,feed_f,out_f,pressure,overflow_t,rake_s,rake_1,rake_2,rake_3,rake_4,out_f_setting,out_c_setting,reward,error,sum_error,not_done
0,2022-01-22 23:20:00,42.50,62.2,635.000,170.0,50.3,0.231,45.0,25.6,26.5,26.1,26.0,150.0,62.5,-3.890119,-3.890119,-3.890119,1
1,2022-01-22 23:21:00,42.40,61.7,536.000,185.0,51.2,0.231,45.0,26.4,26.0,26.4,26.7,150.0,62.5,-4.483084,-4.483084,-7.206168,1
2,2022-01-22 23:22:00,36.90,62.0,450.000,176.0,49.0,0.224,45.0,26.4,25.9,26.6,27.1,150.0,62.5,-4.110529,-4.110529,-9.154846,1
3,2022-01-22 23:23:00,32.20,63.6,551.000,164.0,48.8,0.224,45.0,25.4,26.1,26.9,27.8,150.0,62.5,-5.080288,-5.080288,-11.488680,1
4,2022-01-22 23:24:00,32.60,63.2,731.000,186.0,49.0,0.231,45.0,25.6,26.0,26.9,26.3,150.0,62.5,-4.310486,-4.310486,-12.352562,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,2023-01-19 23:09:00,6.20,55.5,0.434,184.0,32.6,0.109,45.0,19.2,20.4,24.2,20.8,180.0,62.0,-45.509905,-45.509905,-50.000000,1
43643,2023-01-19 23:10:00,6.17,56.4,0.434,175.0,32.4,0.101,45.0,19.1,20.1,24.2,21.0,180.0,62.0,-34.615663,-34.615663,-50.000000,1
43644,2023-01-19 23:11:00,6.25,56.0,0.434,190.0,32.9,0.101,45.0,18.7,20.3,24.5,20.7,180.0,62.0,-39.253472,-39.253472,-50.000000,1
43645,2023-01-19 23:12:00,5.72,56.4,0.434,166.0,32.5,0.101,45.0,18.6,20.0,22.4,20.8,180.0,62.0,-34.548861,-34.548861,-50.000000,1


In [59]:
order = ["feed_c", "out_c", "feed_f", "out_f", "pressure", "rake_1", "rake_2", "rake_3", "rake_4", "error", "sum_error", "out_c_setting", "reward", "out_f_setting", "not_done"]
df = df[order]

In [60]:
df

,feed_c,out_c,feed_f,out_f,pressure,rake_1,rake_2,rake_3,rake_4,error,sum_error,out_c_setting,reward,out_f_setting,not_done
0,42.50,62.2,635.000,170.0,50.3,25.6,26.5,26.1,26.0,-3.890119,-3.890119,62.5,-3.890119,150.0,1
1,42.40,61.7,536.000,185.0,51.2,26.4,26.0,26.4,26.7,-4.483084,-7.206168,62.5,-4.483084,150.0,1
2,36.90,62.0,450.000,176.0,49.0,26.4,25.9,26.6,27.1,-4.110529,-9.154846,62.5,-4.110529,150.0,1
3,32.20,63.6,551.000,164.0,48.8,25.4,26.1,26.9,27.8,-5.080288,-11.488680,62.5,-5.080288,150.0,1
4,32.60,63.2,731.000,186.0,49.0,25.6,26.0,26.9,26.3,-4.310486,-12.352562,62.5,-4.310486,150.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,6.20,55.5,0.434,184.0,32.6,19.2,20.4,24.2,20.8,-45.509905,-50.000000,62.0,-45.509905,180.0,1
43643,6.17,56.4,0.434,175.0,32.4,19.1,20.1,24.2,21.0,-34.615663,-50.000000,62.0,-34.615663,180.0,1
43644,6.25,56.0,0.434,190.0,32.9,18.7,20.3,24.5,20.7,-39.253472,-50.000000,62.0,-39.253472,180.0,1
43645,5.72,56.4,0.434,166.0,32.5,18.6,20.0,22.4,20.8,-34.548861,-50.000000,62.0,-34.548861,180.0,1


In [49]:
#"name": ["feed_c", "out_c", "feed_f", "out_f", "out_f_setting", "pressure", "reward"]

In [61]:
df_1 = df

In [62]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,rake_1,rake_2,rake_3,rake_4,error,sum_error,out_c_setting,reward,out_f_setting,not_done
0,42.50,62.2,635.000,170.0,50.3,25.6,26.5,26.1,26.0,-3.890119,-3.890119,62.5,-3.890119,150.0,1
1,42.40,61.7,536.000,185.0,51.2,26.4,26.0,26.4,26.7,-4.483084,-7.206168,62.5,-4.483084,150.0,1
2,36.90,62.0,450.000,176.0,49.0,26.4,25.9,26.6,27.1,-4.110529,-9.154846,62.5,-4.110529,150.0,1
3,32.20,63.6,551.000,164.0,48.8,25.4,26.1,26.9,27.8,-5.080288,-11.488680,62.5,-5.080288,150.0,1
4,32.60,63.2,731.000,186.0,49.0,25.6,26.0,26.9,26.3,-4.310486,-12.352562,62.5,-4.310486,150.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,6.20,55.5,0.434,184.0,32.6,19.2,20.4,24.2,20.8,-45.509905,-50.000000,62.0,-45.509905,180.0,1
43643,6.17,56.4,0.434,175.0,32.4,19.1,20.1,24.2,21.0,-34.615663,-50.000000,62.0,-34.615663,180.0,1
43644,6.25,56.0,0.434,190.0,32.9,18.7,20.3,24.5,20.7,-39.253472,-50.000000,62.0,-39.253472,180.0,1
43645,5.72,56.4,0.434,166.0,32.5,18.6,20.0,22.4,20.8,-34.548861,-50.000000,62.0,-34.548861,180.0,1


## 标准化

In [63]:
df_zscore = df_1.copy()
done_o = df_zscore["not_done"]
df_zscores = (df_zscore - df_zscore.mean()) / df_zscore.std()
df_zscores['not_done'] = done_o
df_zscores

,feed_c,out_c,feed_f,out_f,pressure,rake_1,rake_2,rake_3,rake_4,error,sum_error,out_c_setting,reward,out_f_setting,not_done
0,1.714893,0.907106,0.533320,0.992437,0.965298,0.914100,0.945952,0.543702,0.903281,NaN,1.880884,4.436815,NaN,-0.015592,1
1,1.701371,0.693350,0.094678,1.596362,1.103498,1.020305,0.880635,0.583313,0.995192,NaN,1.652677,4.436815,NaN,-0.015592,1
2,0.957696,0.821604,-0.286365,1.234007,0.765676,1.020305,0.867572,0.609721,1.047712,NaN,1.518572,4.436815,NaN,-0.015592,1
3,0.322192,1.505624,0.161139,0.750867,0.734965,0.887548,0.893699,0.649333,1.139623,NaN,1.357960,4.436815,NaN,-0.015592,1
4,0.376277,1.334619,0.958669,1.636623,0.765676,0.914100,0.880635,0.649333,0.942671,NaN,1.298509,4.436815,NaN,-0.015592,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43642,-3.193365,-1.957226,-2.278269,1.556100,-1.752634,0.064459,0.149085,0.292827,0.220514,NaN,-1.292344,-0.225382,NaN,1.222727,1
43643,-3.197421,-1.572465,-2.278269,1.193745,-1.783346,0.051183,0.109894,0.292827,0.246775,NaN,-1.292344,-0.225382,NaN,1.222727,1
43644,-3.186604,-1.743470,-2.278269,1.797670,-1.706568,-0.001919,0.136021,0.332438,0.207384,NaN,-1.292344,-0.225382,NaN,1.222727,1
43645,-3.258267,-1.572465,-2.278269,0.831391,-1.767990,-0.015195,0.096831,0.055156,0.220514,NaN,-1.292344,-0.225382,NaN,1.222727,1


In [64]:
df_zscore.std()

feed_c             7.395700
out_c              2.339114
feed_f           225.696621
out_f             24.837538
pressure           6.512303
rake_1             7.532594
rake_2             7.654974
rake_3             7.573498
rake_4             7.616074
error                   NaN
sum_error         14.530908
out_c_setting      0.107246
reward                  NaN
out_f_setting     24.226387
not_done           0.061738
dtype: float64

In [26]:
df_zscore_mean = pd.DataFrame(df_zscore.mean()).stack().unstack(0)
df_zscore_mean.to_csv("df_thickener_zscore_mean.csv", index=False)

df_zscore_std = pd.DataFrame(df_zscore.std()).stack().unstack(0)
df_zscore_std.to_csv("df_thickener_zscore_std.csv", index=False)

## 数据集两两绑定（state和next_state）

In [27]:
df_zscores_single = [x for x in range(0, len(df_zscores), 2)]
len(df_zscores_single)

4978

## 划分验证集和训练集（差集）-------两两绑定

In [30]:
df_zscores_single_eval = random.sample(df_zscores_single, 1000)

In [31]:
len(df_zscores_single_eval)

1000

In [32]:
df_zscores_single_train = list(set(df_zscores_single) - set(df_zscores_single_eval))

In [33]:
len(df_zscores_single_train)

3978

In [34]:
data = df_zscores.values

In [35]:
print(data[0])
data[0][0:3]

[-2.66329984  1.88633454 -2.34642718 -0.97539465  1.0511237  -1.0148855
 -1.03421695  1.08497778  0.84962664 -1.03878587  0.        ]


array([-2.66329984,  1.88633454, -2.34642718])

In [39]:
state_dim = 8
action_dim = 1

In [40]:
replay_buffer_train = ReplayBuffer(state_dim, action_dim, device="cpu")
replay_buffer_eval = ReplayBuffer(state_dim, action_dim ,device="cpu")

In [ ]:
for t in range(0, len(data)-1 , 2):   
    print(t)
    if t in df_zscores_single_train:
        state = data[t][0:-3]
        action = data[t][-2]
        next_state = data[t+1][0:-3]
        reward = data[t+1][-3]
        done = data[t+1][-1]
        replay_buffer_train.add(state, action, next_state, reward, done)
        print("replay_buffer_train_add", t)
    elif t in df_zscores_single_eval:
        state = data[t][0:-3]
        action = data[t][-2]
        next_state = data[t+1][0:-3]
        reward = data[t+1][-3]
        done = data[t+1][-1]
        replay_buffer_eval.add(state, action, next_state, reward, done)
        print("replay_buffer_eval_add", t)
    else:
        pass

In [109]:
# # 训练集
# for t in range(len(data_train[:-1])):
#     state = data[t][0:-4]
#     action = data[t][-2]
#     next_state = data[t+1][0:-4]
#     reward = data[t+1][-3]
#     done = data[t][-1]
#     replay_buffer_train.add(state, action, next_state, reward, done)
# # 验证集
# for t in range(len(data_eval[:-1])):
#     state = data[t][0:-4]
#     action = data[t][-2]
#     next_state = data[t+1][0:-4]
#     reward = data[t+1][-3]
#     done = data[t][-1]
#     replay_buffer_eval.add(state, action, next_state, reward, done)

In [44]:
buffer_name = "replay_buffer_change_8_1"
replay_buffer_train.save(f"./buffers/{buffer_name}_train")
replay_buffer_eval.save(f"./buffers/{buffer_name}_eval")